In [1]:
import os
import pandas as pd

Useful Source: https://www.youtube.com/watch?v=9MyIABer_NY&list=PLuY0WH4mYH1NvZMss0oKonajGdNl6IvuT

### 1. Node file

In [2]:
# Load zip code coordinates data
data_dir = "./data/"

data_path = os.path.join(data_dir, "uszips.csv")
zipcodes = pd.read_csv(data_path)
zipcodes = zipcodes[["zip", "lat", "lng"]]
zipcodes.rename(columns={"lat": "x", "lng": "y"}, inplace=True)
zipcodes

,zip,x,y
0,601,18.18027,-66.75266
1,602,18.36075,-67.17541
2,603,18.45744,-67.12225
3,606,18.16585,-66.93716
4,610,18.29110,-67.12243
...,...,...,...
33782,99923,55.98043,-130.03803
33783,99925,55.55398,-132.96276
33784,99926,55.12617,-131.48928
33785,99927,56.33305,-133.60044


In [3]:
# Load OD matrix
od_matrix = pd.read_csv("./od_matrix.csv", index_col=0)
od_matrix


,6390,10001,10002,10003,10004,10005,10006,10007,10009,10010,...,14892,14893,14894,14895,14897,14898,14901,14903,14904,14905
origin_zcta,,,,,,,,,,,,,,,,,,,,,
6390,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10001,1,1253,1354,1496,126,269,122,203,1392,801,...,3,0,2,1,0,0,5,1,3,3
10002,0,61,1574,143,9,27,3,21,413,60,...,0,0,0,0,0,0,0,0,0,0
10003,1,343,664,1122,58,126,31,66,875,451,...,0,0,0,0,0,0,0,0,0,0
10004,0,211,553,350,124,226,87,93,395,214,...,2,0,0,1,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14898,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,2,15,2,0,0,2
14901,0,1,3,1,0,0,0,0,0,0,...,131,1,116,3,0,15,823,474,1066,717
14903,0,0,0,0,0,0,0,0,0,0,...,18,0,40,3,1,9,251,297,345,258


In [4]:
# Extract unique ZCTAs (nodes)
zctas = list(od_matrix.index)
print(len(zctas))

1826


In [5]:
node_data = zipcodes[zipcodes['zip'].isin(zctas)]
node_data.dtypes

for zcta in zctas:
    if int(zcta) not in list(node_data["zip"]):
        print(zcta)

# good! none missing

In [6]:
def generate_node_file_from_zip(zipcodes, zctas, type="zcta", output_file="my_nodes.nod.xml"):
    # Filter ZIP codes to include only the ZCTAs in the OD matrix
    node_data = zipcodes[zipcodes['zip'].isin(zctas)]

    # Create the node file
    with open(output_file, "w") as f:
        f.write("<nodes>\n")
        for _, row in node_data.iterrows():
            f.write(f'    <node id="{int(row["zip"])}" x="{row["x"]}" y="{row["y"]}" type="{type}"/>\n')
        f.write("</nodes>\n")

    print(f"Node file '{output_file}' created successfully!")


In [7]:
generate_node_file_from_zip(zipcodes, zctas, type="zcta", output_file="my_nodes.nod.xml")

Node file 'my_nodes.nod.xml' created successfully!


### 2. Edge file

In [8]:
def generate_edge_file(od_matrix, edge_type="none", output_file="my_edges.edg.xml"):

    # Create edge data from non-zero OD matrix entries
    # edges = []
    # for origin in od_matrix.index:
    #     for destination in od_matrix.columns:
    #         if od_matrix.loc[origin, destination] > 0:  # Non-zero OD value
    #             edge_id = f"{origin}to{destination}"
    #             edges.append({"from": origin, "to": destination, "id": edge_id, "type": edge_type})

    # Create edge data from OD matrix entries
    edges = []
    for origin in od_matrix.index:
        for destination in od_matrix.columns:
            edge_id = f"{origin}to{destination}"
            edges.append({"from": origin, "to": destination, "id": edge_id, "type": edge_type})

    
    # Write to XML file
    with open(output_file, "w") as f:
        f.write("<edges>\n")
        for edge in edges:
            f.write(
                f'    <edge from="{edge["from"]}" to="{edge["to"]}" id="{edge["id"]}" type="{edge["type"]}"/>\n'
            )
        f.write("</edges>\n")
    
    print(f"Edge file '{output_file}' created successfully!")


In [9]:
generate_edge_file(od_matrix, edge_type="none", output_file="my_edges.edg.xml")

Edge file 'my_edges.edg.xml' created successfully!


### 3. Edge-type file

In [10]:
# id, priority (higher, more priority), numLanes, speed
# for example: <type id:”3L45” priority=”3” numLanes=”3” speed=”45”/>
# TODO

### 4. Network file

In [11]:
# Comment the line below out and run
# Or comment the line below out and copy and paste in the terminal without !
# ! netconvert --node-files my_nodes.nod.xml --edge-files my_edges.edg.xml -t my_type.type.xml -o my_net.net.xml

### 5. Routing file

In [12]:
def generate_routing_file(od_matrix, output_file="my_routes.rou.xml"):
    vehicle_count = 0  # Unique vehicle ID counter

    with open(output_file, "w") as f:
        # Write header
        f.write("<routes>\n")
        
        # Define vehicle types
        f.write('    <vType id="Human" accel="0.5" decel="0.5" length="0.6" maxSpeed="5.0" sigma="0.5"/>\n')
        f.write('    <vType id="Cyclist" accel="1.0" decel="1.0" length="2.0" maxSpeed="15.0" sigma="0.2"/>\n')
        f.write('    <vType id="Car" accel="2.5" decel="4.5" length="4.5" maxSpeed="120.0" sigma="0.2"/>\n')
        f.write('    <vType id="Bus" accel="1.5" decel="3.5" length="12.0" maxSpeed="80.0" sigma="0.3"/>\n')
        f.write('    <vType id="Truck" accel="1.0" decel="3.5" length="18.0" maxSpeed="90.0" sigma="0.3"/>\n')

        # Generate routes and vehicles from OD matrix
        for origin in od_matrix.index:
            for destination in od_matrix.columns:
                flow = od_matrix.loc[origin, destination]
                if flow > 0:  # If there's traffic flow
                    # Define a route
                    route_id = f"route_{origin}_to_{destination}"
                    f.write(f'    <route id="{route_id}" edges="{origin}to{destination}"/>\n')
                    
                    # Add vehicles for this route
                    for i in range(int(flow)):  # Assuming 'flow' represents the number of vehicles
                        vehicle_id = f"veh_{vehicle_count}"
                        f.write(f'    <vehicle id="{vehicle_id}" type="Human" route="{route_id}" depart="{vehicle_count * 2}"/>\n')
                        vehicle_count += 1

        # Write footer
        f.write("</routes>\n")
    
    print(f"Routing file '{output_file}' created successfully!")

In [13]:
generate_routing_file(od_matrix, output_file="my_routes.rou.xml")

Routing file 'my_routes.rou.xml' created successfully!
